# Spark DataFrame

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.\
    master("local[*]").\
    config("spark.executor.memory", "4g").\
    config("spark.driver.memory", "4g").\
    config("spark.ui.showConsoleProgress", "false").\
    appName("DataFrame").\
    getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)

### 数据读取

JSON 文件可以直接使用 PySpark 读取：

In [ ]:
df1 = spark.read.json("data/lec14-danmu-144541892.json", multiLine=True)
df1.count()

In [ ]:
df2 = spark.read.json("data/lec14-danmu-144541943.json", multiLine=True)
df2.count()

对于 JSON 文件，PySpark 中一个非常实用的功能是将一列文件作为输入，其返回的结果会自动将所有文件合并：

In [ ]:
jsons = ["data/lec14-danmu-144541892.json", "data/lec14-danmu-144541943.json"]
df3 = spark.read.json(jsons, multiLine=True)
df3.count()

我们将所有剧集的弹幕文件统一进行读取：

In [ ]:
# cids = [144541892, 144541943, 160377038, 148952771, 150894103, 153392221, 156629080, 159982308, 162395026]
cids = [144541892, 144541943, 160377038, 148952771, 150894103]
jsons = [f"data/lec14-danmu-{cid}.json" for cid in cids]
jsons

In [ ]:
df = spark.read.json(jsons, multiLine=True)
df.count()

PySpark 还支持许多其他类型的数据，如常用的 CSV 等。请参考[官方文档](https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html#Getting-Data-in/out)。

### 查看数据和结构

使用 `show()` 函数可以打印数据的前若干行，注意数据的合并顺序与输入的文件列表不一定一致：

In [ ]:
df.show()

In [ ]:
df.show(n=5)

读取得到的对象其类型为 `DataFrame`：

In [ ]:
type(df)

用 `printSchema()` 可以打印出各变量的类型：

In [ ]:
df.printSchema()

`DataFrame` 本质上是对 RDD 的一种更高层的封装。我们可以直接取出 `DataFrame` 背后的 RDD：

In [ ]:
rdd = df.rdd
print(rdd)
print()
print(type(rdd))
print()
print(rdd.getNumPartitions())
print()
print(rdd.toDebugString().decode(encoding="utf-8"))

该 RDD 的元素类型为 `Row`：

In [ ]:
r1 = rdd.first()
print(r1)
print()
print(type(r1))

`Row` 对象类似于字典，可以用 key 取出其中的变量值。另一种方法是直接取属性：

In [ ]:
print(r1["cid"])
print()
print(r1.cid)

也可以直接转换为字典：

In [ ]:
r1.asDict()

PySpark 的 `DataFrame` 也可以转换为 Pandas 的 `DataFrame`，但注意不要轻易对完整数据转换！必要时可以先使用 `limit()` 限制行数：

In [ ]:
df_pandas = df.limit(10).toPandas()
df_pandas

### 数据存储

`DataFrame` 可以用特定的格式保存到硬盘上，如 CSV：

In [ ]:
# 在 Windows 系统上运行可能会报错
df100 = df.limit(100)
df100.write.csv("data/lec14-output.csv", header=True)

### 选择操作

选择列和选择行：

In [ ]:
df.select("content").show(n=5)

In [ ]:
df.drop("content").show(n=5)

In [ ]:
df.filter(df.mode != 1).show(n=5)

`filter()` 可以替换为 `where()`，判断条件也可以用字符串表示（直接使用变量名）：

In [ ]:
df.where(df.mode != 1).show(5)
df.where("mode != 1").show(5)

不同的操作可以串起来：

In [ ]:
df.select("content", "mode").where("mode != 1").show(5)

`distinct()` 可以用来去重：

In [ ]:
df.select("date").distinct().show(n=5)

df.select("date").distinct().count()

### 数据变换

在 `DataFrame` 中可以根据已有的列变换得到新的列：

In [ ]:
df.select("id").withColumn("id_mod", df.id % 10).show()

PySpark 也提供了很多对列进行变换的函数，定义在 `pyspark.sql.functions` 中：

In [ ]:
from pyspark.sql.functions import date_format, from_unixtime

df.select("post_time").withColumn("post_time_formatted", from_unixtime(df.post_time)).show()

In [ ]:
df.select("date").distinct().withColumn("date_formatted", date_format("date", "yyyy.MM.dd")).\
    withColumn("day_of_week", date_format("date", "E")).show()

### 连接操作

有时需要将两张或以上的表的信息进行连接，此时可以使用 `join()` 函数。

首先读取一份视频信息的 `DataFrame`：

In [ ]:
video_info = spark.read.json("data/lec14-video-data.json", multiLine=True)
video_title = video_info.select("cid", "title")
video_title.show()

我们希望在查看弹幕的时候，能够知道对应的剧集标题。此时以两张表的 `cid` 作为连接条件：

In [ ]:
df.join(video_title, df.cid == video_title.cid, "inner").drop(df.id).drop(video_title.cid).show()

### SQL 操作

更灵活的数据操作可以使用 SQL 语句实现，例如汇总操作。简单统计每集有多少弹幕：

In [ ]:
# 将 df 注册为一张表
df.createOrReplaceTempView("danmu")

danmu_stats = spark.sql("select cid, count(*) as num_of_danmu from danmu group by cid")
danmu_stats.show()

连接操作：

In [ ]:
danmu_stats.createOrReplaceTempView("danmu_stats")
video_title.createOrReplaceTempView("video_title")

spark.sql("""select * from danmu_stats inner join video_title
             where danmu_stats.cid=video_title.cid
             order by title""").show()

更多 SQL 语句的操作可以参考[这个教程](https://www.w3school.com.cn/sql/index.asp)。